# MDA 2021
-----------------------------------------------------------------

## Setup
--------------------------------------------------

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

You should consider upgrading via the '/home/amin/.local/share/virtualenvs/HW-QXNApxFg/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/amin/.local/share/virtualenvs/HW-QXNApxFg/bin/python -m pip install --upgrade pip' command.
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


Now we authenticate a Google Drive client to processing data

**Make sure to follow the interactive instructions.**

In [2]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


## Check and extract data
--------------------------------------------------

In [ ]:
!ls '/content/drive/My Drive/Colab Notebooks/MDA/HW3/'

 checkstatus.csv   MDA_2021.ipynb       SystemID.csv
 CompanyID.csv	   Sample_Data.zip      Traffic.csv
 Data.zip	   Sample_Traffic.csv  'پروژه MDA2021.pdf'


In [2]:
# !unzip "/content/drive/My Drive/Colab Notebooks/MDA/HW3/Sample_Data.zip" -d "/content/drive/My Drive/Colab Notebooks/MDA/HW3"
!unzip "./Sample_Data.zip" -d "./"

Archive:  ./Sample_Data.zip
  inflating: ./Sample_Traffic.csv    


the cells above, extract data which is in '/content/drive/My Drive/Test' to /content/drive/My Drive/Test/Traffic.csv  

## Initializing Spark and read data
--------------------------------------------------

In [1]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,TimestampType
from pyspark.sql.functions import col,current_timestamp,to_date,hour,dayofweek
spark = SparkSession \
    .builder \
    .appName("Spark_Processor") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

schema = StructType([ \
        StructField("DEVICE_CODE", IntegerType(), True), 
        StructField("SYSTEM_ID",IntegerType(),True), \
        StructField("ORIGINE_CAR_KEY",StringType(),True), \
        StructField("FINAL_CAR_KEY",StringType(),True), \
        StructField("CHECK_STATUS_KEY", IntegerType(), True), \
        StructField("COMPANY_ID", StringType(), True), \
        StructField("PASS_DAY_TIME", TimestampType(), True)
    ])

21/12/28 17:45:36 WARN Utils: Your hostname, amin-X556UQK resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface wlp3s0)
21/12/28 17:45:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/28 17:45:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/28 17:45:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/12/28 17:45:38 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


<h3 dir="rtl">
 در این قسمت، لطفا آدرس فایل 
csv
 را به جای آدرس قرار داده شده جایگذاری کنید.  
  
</h3> 

In [2]:
# df=spark.read.csv('/content/drive/My Drive/Colab Notebooks/MDA/HW3/Sample_Traffic.csv',header=True,schema=schema)
df=spark.read.csv('Sample_Traffic.csv',header=True,schema=schema)
# df.select('DEVICE_CODE').distinct().count()
df.show(5)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|     200501|       81|       10477885|     10477885|               5|       161|2021-06-01 03:54:39|
|        155|       81|       87625017|     87625017|               5|       161|2021-06-01 04:14:21|
|     631757|       81|        8652928|      8652928|               5|       161|2021-06-01 03:58:57|
|     631757|       81|        8548123|      8548123|               5|       161|2021-06-01 04:01:38|
|     631757|       81|       24715264|     24715264|               5|       161|2021-06-01 03:56:57|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
only showing top 5 rows



<h3 dir="rtl">
 با استفاده از یک بار فرایند 
 map
 ، هر ردیف از 
 df
 را به صورت کلید پلاک خودرو، و زمان عبور از مسیر در می‌آوریم. مقدار این کلید را نیز برابر کد دستگاه (
     که نمایانگر مسیر عبوری است 
 ) 
 می‌گذاریم. 
 rdd
 به دست آمده به ازای هر مسیر در هر روز، یک ردیف خواهد داشت. 
</h3> 


In [3]:
rdd = df.rdd.map(lambda x: ((x['FINAL_CAR_KEY'], x['PASS_DAY_TIME'].date()),  x['DEVICE_CODE']))
rdd.take(5)

[(('10477885', datetime.date(2021, 6, 1)), 200501),
 (('87625017', datetime.date(2021, 6, 1)), 155),
 (('8652928', datetime.date(2021, 6, 1)), 631757),
 (('8548123', datetime.date(2021, 6, 1)), 631757),
 (('24715264', datetime.date(2021, 6, 1)), 631757)]

<h3 dir="rtl">
 اگر 
 rdd
 بالا را به کمک کلیدش 
 group_by
 کنیم، برای هر ماشین در هر روز خاص، تمام مسیر‌هایی که عبور کرده در یک لیست خواهیم داشت. البته به دلیل ماهیت مسئله 
 frequent_itemsets
 ، 
 به جای لیست از
 set
 استفاده کرده‌ام، تا مسیر‌های تکراری نداشته باشیم. هرچند می‌توان مسئله را به گونه‌ای تعریف کرد که ترتیب و یا تعداد تکرار مسیرها تاثیر گذار باشد، اما با توجه به خروجی‌ای که ما می‌خوایم، این فرض چیزی از کلیت مسئله نمی‌کاهد.  
</h3> 


In [31]:
path_rdd = rdd.groupByKey().mapValues(set)
path_rdd.take(5)

[(('69939810', datetime.date(2021, 6, 1)), {206602, 631830, 900233}),
 (('11046172', datetime.date(2021, 6, 1)), {206602}),
 (('29077699', datetime.date(2021, 6, 1)),
  {119,
   128,
   206602,
   208602,
   210602,
   631367,
   631763,
   631829,
   900135,
   900233,
   900234,
   900240,
   900246,
   900266,
   22010111,
   22010123}),
 (('40682798', datetime.date(2021, 6, 1)), {206602}),
 (('48823778', datetime.date(2021, 6, 1)), {206602})]

In [235]:
path_rdd.count()

1520404

# ب)

## Step by Step

<h3 dir="rtl">
 برای برخی از کاربردها، نیازی به نگه داشتن پلاک و روز نداریم. بنابرین در یک 
 RDD
 دیگر آن‌ها را بدون پلاک و روز نگه می‌داریم.  
</h3> 


In [242]:
import numpy as np
import math

path_rdd_without_key = path_rdd.map(lambda x: x[1])
# path_rdd_without_key.take(10)

In [243]:
# sample_path = path_rdd.take(3)[2]
# sample_path
sample_path = {206602, 631830, 900233}

<h3 dir="rtl">
 تابع زیر، به عنوان ورودی یک مسیر می‌گیرد. سپس با اشتراک گرفتن بین این مسیر و مسیر نمونه، و تقسیم کردن اندازه‌ی اشتراک به ضرب اندازه‌ی دو مسیر، فاصله‌ی کسینوسی بین مسیر نمونه و آن مسیر را محاسبه می‌کند و خروجی می‌دهد. 
</h3> 


In [244]:
def similarity(xx):
    x = xx[1] 
    # print('sample_path is: ', sample_path)
    # print('x is: ', x)

    distance = len(sample_path.intersection(x))  / math.sqrt((len(x)*len(sample_path)) ) 
    # print(distance)

    return (xx, distance)


<h3 dir="rtl">
 در این مرحله ۵ مسیر با بیشترین شباهت را نشان می‌دهیم. در هر 
 tuple
 دو مقدار اول پلاک ماشین و تاریخ هستد و درایه‌ی بعدی مسیر طی شده. درایه‌ی آخر نیز میزان شباهت است. 
</h3> 


In [246]:
path_rdd.map(similarity).sortBy(lambda x: x[1], ascending=False).take(5)

[((('69939810', datetime.date(2021, 6, 1)), {206602, 631830, 900233}), 1.0),
 ((('11888001', datetime.date(2021, 6, 1)), {206602, 631830}),
  0.8164965809277261),
 ((('9538558', datetime.date(2021, 6, 1)), {206602, 900233}),
  0.8164965809277261),
 ((('17644738', datetime.date(2021, 6, 1)), {206602, 631830}),
  0.8164965809277261),
 ((('8807618', datetime.date(2021, 6, 1)), {206602, 900233}),
  0.8164965809277261)]

# ج)

<h3 dir="rtl">
 تابع زیر به عنوان ورودی تعداد توابع هش را می‌گیرد، و به همان تعداد بردارهایی از ۱ و -۱ می‌سازد. در این مرحله، این بردارها را به صورت دیکشنری ساختیم. زیرا در مراحل بعدی، برای محاسبه‌ی ضرب داخلی کار ما خیلی راحت تر می‌شود و سرعت محاسبه نیز بالاتر می‌رود. 
</h3> 


In [247]:
import random
def generate_vectors(n):
    all_path = dict.fromkeys(path_rdd_without_key.flatMap(lambda x: x).distinct().collect(), 1)


    vectors = [None]*n
    for i in range(n):
        v = all_path.copy()
        for key in all_path:
            v[key] = [1,-1][random.randint(0, 1)]

        vectors[i] = v

    return vectors

<h3 dir="rtl">
 متغیر 
 n
 تعداد توابع هش را نشان می‌دهد. متغیرهای 
 AND 
 و
 OR 
 نیز به ترتیب ضخامت نوار و تعداد نوار‌ها را نشان می‌دهند. به وضوح داریم:
 AND . OR = n 

 همچنین در این قسمت تابع بالا را نیز استفاده می‌کنیم تا بردار‌های رندم را تولید کنیم. 
</h3> 


In [248]:
n = 500
AND = 25
OR = 20
vectors = generate_vectors(n)

<h3 dir="rtl">
 تابع به عنوان ورودی لیستی از مسیر‌ها دریافت می‌کند و در خروجی 
 signature
 آن را به دست می‌آورد. برای این کار، به ازای هر بردار، باید ضرب داخلی بردار متناظر با این لیست، در تمام بردار‌های رندم مرحله‌ی قبلی را به دست آوریم. از آنجایی که لیست‌ها به نسبت تمام جاده‌های ممکن کوچک‌اند، در این جا تنها درایه‌هایی از بردار‌های رندم را می‌گیریم که در لیست ما وجود دارند و به سادگی ضرب داخلی را به دست می‌آوریم. سپس اگر ضرب داخلی مثبت شد، نتیجه‌ی آن هش را ۱ و در غیر این صورت صفر در نظر می‌گیریم.  
</h3> 


In [249]:
def min_hash(path):
    column = path[1] 
    n = len(vectors)
    signature = [1]*n
    for i, v in enumerate(vectors):
        # print('v is: ', v)
        sum = 0
        for key in column:
            # print('key is: ', key)
            # print('v[key] = ', v[key])
            sum += v[key]

        if sum < 0:
            signature[i] = 0
            
    return (path[0], path[1], signature)


<h3 dir="rtl">
 حال با استفاده از تابع بالا و یک مرحله
 map
 برای هر مسیر 
 sinatureش 
 را حساب می‌کنیم.  
</h3> 


In [250]:
signatures = path_rdd.map(min_hash)

<h3 dir="rtl">
  تابع زیر نیز به عنوان ورودی یک 
  signature
  می‌گیرد و در خروجی، به ازای هر بند از این 
signature
یک 
tuple
تولید می‌کند، که تمام اطلاعات مربوط به شماره‌ی هش، مقدار هش و اطلاعات اولیه را نگه می‌دارد. دلیل نگه داشتن این اطلاعت، استفاده از آن‌ها در مراحل بعدی برای نشان دادن نتیجه است. ترکیب شماره‌ی هش و مقدار هش، به ما یک سبد خاص از یک هش را می‌دهد. در چند مرحله‌ی بعد از این نکته‌ استفاده می‌کنیم.
</h3> 


In [251]:
def hash_band(x):
    signature = x[2]
    # print(signature)
    output = []
    for i in range(OR):
        band = signature[i*AND:(i+1)*AND]
        # print(band)
        band_binary = ''.join(str(digit) for digit in band)
        band_hash = int(band_binary, 2)
        # print(band_hash)
        output.append(
            (x[0], i, band_hash, x[1], x[2])
        )

    return output

<h3 dir="rtl">
  حال به کمک تابعی که توضیح دادیم، 
  tupleهای 
  گفته شده را به دست می‌آوریم.  
</h3> 


In [ ]:
hash_baskets = signatures.flatMap(hash_band)
# hash_baskets.take(2)

<h3 dir="rtl">
 در این مرحله مراحل بالا را برای مسیر فرضی نمونه خود نیز انجام می‌دهیم. 
</h3> 


In [253]:
sample_signature = min_hash(('sample car', sample_path))
# print(sample_signature)
sample_baskets = hash_band(sample_signature)
# print(sample_baskets[0])


('sample car', {900233, 206602, 631830}, [0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0

<h3 dir="rtl">
 در این مرحله که مهم ترین قسمت کد است، کاندید‌های نهایی را به دست می‌آوریم. برای این کار، به این نکته‌ دقت کنید که ترکیب شماره‌ی هش و مقدار هش، به ما یک سبد خاص از یک هش را می‌دهد. حال به ازای هرکدام از هش‌ها، تنها سبد‌هایی را نگه می‌داریم که هش مسیر فرضی ما نیز مقدارش در آن سبد قرار می‌گیرد. این یعنی تنها مسیر‌هایی باقی می‌مانند که حداقل در یکی از نوار‌ها هشی برابر با هش مسیر فرضی در آن نوار داشته‌اند. سپس تعداد چنین کاندید‌هایی را می‌شماریم. 
</h3> 


In [254]:
candidates = hash_baskets.filter(lambda x: any([ (x[1]==i) and (x[2] == sample_baskets[i][2]) for i in range(OR)]) )
print('candidates count: ', candidates.count())

candidates count:  24


In [231]:
candidates.take(1)

[(('69939810', datetime.date(2021, 6, 1)),
  0,
  31125503,
  {206602, 631830, 900233},
  [1,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   0,
   1,
   0,
   0,
   1,
   1,
   0,
   1,
   0,
   0,
   1,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   1,
   0,
   1,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   0,
   0,
   1,
   1,
   0,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,


<h3 dir="rtl">
 در این مرحله، به کمک 
 groupBy
 و 
 map
 مسیر‌های تکراری را حذف می‌کنیم. مسیر‌های باقی مانده (که کاندید هستند) حتما متمایز‌اند. در بلاک بعدی نیز تعداد آن‌ها را می‌شماریم.  
</h3> 


In [255]:
candidates_unique = candidates.map(lambda x: (x[0], (x[1],x[2],x[3],x[4]))).groupByKey().mapValues(lambda x: list(x)[0]).collect()

In [256]:
len(candidates_unique)

5

<h3 dir="rtl">
در
 دو بلاک بعدی، نتیجه‌ را به صورتی که قابل فهم باشد در فایل
res.txt
ذخیره می‌کنیم. 
  
</h3> 


In [257]:
t = candidates.map(lambda x: (x[0], x[3])).groupByKey().mapValues(lambda x: list(x)[0]).collect()


In [258]:

import sys
original_stdout = sys.stdout
with open('res.txt', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(t)
    sys.stdout = original_stdout # Reset the standard output to its original value



# پارامتر‌ها و نتایج

<h3 dir="rtl">
 طبیعتا هرچه تعداد 
 hyperplane
 ها را بیشتر کنیم، می‌توانیم میزان 
 False Negative
 و
 False Positive
 را کاهش دهیم. با بازی کردن با پارامتر‌ها، بهترین نتیجه را با پارامتر‌هایی گرفتم که در این صفحه می‌بینید (
     یعنی
     AND = 25
OR = 20). همانطوری که گفتم نتایج به دست آمده نیز در 
فایل 
res.txt
پیوست شده است. 
     
</h3> 
